In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import helpers.processing_helpers as ph

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor

In [2]:
df_dev = pd.read_csv("./dataset/development.csv")

In [3]:
noise_indexes = [0,7,12,15,16,17]
acc_idxs = [1,2,3,4,5,6,8,9,10,11,13,14]
features = ["pmax", "negpmax", 'area', 'tmax', 'rms']

sensors_removed = df_dev.drop(columns=ph.get_column_names(features, noise_indexes))
sensors_removed = sensors_removed.drop(columns=ph.get_column_names(['tmax', 'rms'], acc_idxs))

In [4]:
negpmax_clms = sensors_removed[[f'negpmax[{i}]' for i in acc_idxs]]
df_above_zero = negpmax_clms[(negpmax_clms > 0).any(axis=1)]
df_zero = sensors_removed.drop(df_above_zero.index.values)

In [5]:
joined_negpmax = pd.concat([df_zero[clmn] for clmn in negpmax_clms])
df_zero = df_zero.drop(joined_negpmax[joined_negpmax < -100].index)

In [6]:
X, y = ph.insert_zeros(df_zero, acc_idxs, 17)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, shuffle=True, random_state=42)

In [7]:
reg = RandomForestRegressor(100, random_state=42)
reg.fit(X_train , y_train)

In [ ]:
med = ph.mean_euclid_dist(y_valid, reg.predict(X_valid))